In [1]:
from base_dataset import BaseDataset, MattingTransform

import numpy as np
from PIL import Image
import cv2

/home/ubuntu/miniconda3/envs/hrnet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = '/home/ubuntu/data/workspace/deeplabv3_plus/people_segmentation'
train_transform = MattingTransform()
ds = BaseDataset(root_dir, image_dir="images", mask_dir="masks", transform=train_transform)
# ds = BaseDataset(root_dir)

5678


In [3]:
sample = ds[4]

In [4]:
fn, img, trimap, mask = sample

In [5]:
img.shape

torch.Size([3, 512, 512])

In [6]:
np.unique(img)

array([-1.        , -0.99215686, -0.9843137 , -0.9764706 , -0.96862745,
       -0.9607843 , -0.9529412 , -0.94509804, -0.9372549 , -0.92941177,
       -0.92156863, -0.9137255 , -0.90588236, -0.8980392 , -0.8901961 ,
       -0.88235295, -0.8745098 , -0.8666667 , -0.85882354, -0.8509804 ,
       -0.84313726, -0.8352941 , -0.827451  , -0.81960785, -0.8117647 ,
       -0.8039216 , -0.79607844, -0.7882353 , -0.78039217, -0.77254903,
       -0.7647059 , -0.75686276, -0.7490196 , -0.7411765 , -0.73333335,
       -0.7254902 , -0.7176471 , -0.70980394, -0.7019608 , -0.69411767,
       -0.6862745 , -0.6784314 , -0.67058825, -0.6627451 , -0.654902  ,
       -0.64705884, -0.6392157 , -0.6313726 , -0.62352943, -0.6156863 ,
       -0.60784316, -0.6       , -0.5921569 , -0.58431375, -0.5764706 ,
       -0.5686275 , -0.56078434, -0.5529412 , -0.54509807, -0.5372549 ,
       -0.5294118 , -0.52156866, -0.5137255 , -0.5058824 , -0.4980392 ,
       -0.49019605, -0.4823529 , -0.47450978, -0.46666664, -0.45

In [ ]:
mask.shape

In [ ]:
trimap.shape

In [ ]:
fn

In [ ]:
Image.fromarray(np.array(np.squeeze(np.transpose(img, (1, 2, 0))).numpy() * 255.0, dtype=np.uint8))

In [ ]:
Image.fromarray(np.array(np.squeeze(np.transpose(mask, (1, 2, 0))).numpy() * 255.0, dtype=np.uint8))

In [ ]:
Image.fromarray(np.array(np.squeeze(np.transpose(trimap, (1, 2, 0))).numpy() * 255.0, dtype=np.uint8))

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from scipy.ndimage import morphology
import numpy as np
from PIL import Image
import cv2

In [ ]:
torch_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [ ]:
class ImagesDataset(Dataset):
    def __init__(self, root, transform=None, w=1024, h=576):
        self.root = root
        self.transform = transform
        self.tensor = transforms.Compose([transforms.ToTensor()])
        self.w = w
        self.h = h
        self.imgs = sorted([os.path.join(self.root, 'images', img) for img in os.listdir(os.path.join(self.root, 'images'))])
        self.alphas = sorted([os.path.join(self.root, 'masks', aph) for aph in os.listdir(os.path.join(self.root, 'masks'))])
        print(len(self.imgs))
        assert len(self.imgs) == len(self.alphas), 'the number of dataset is different, please check it.'


    def getTrimap(self, alpha):
        fg = np.array(np.equal(alpha, 255).astype(np.float32))
        unknown = np.array(np.not_equal(alpha, 0).astype(np.float32))  # unknown = alpha > 0
        unknown = unknown - fg
        unknown = morphology.distance_transform_edt(unknown == 0) <= np.random.randint(10, 20)
        trimap = fg
        trimap[unknown] = 0.5
        return trimap[:, :, :1]

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = cv2.imread(self.imgs[idx])
        alpha = cv2.imread(self.alphas[idx])
        # h, w, c = img.shape
        # rh = 512
        # rw = int(w / h * 512)
        # rh = rh - rh % 32 #512
        # rw = rw - rw % 32 #896    1024 or 576 %32==0
        img = cv2.resize(img, (self.w, self.h))
        alpha = cv2.resize(alpha, (self.w, self.h))
        trimap = self.getTrimap(alpha)
        if self.transform:
            img = self.transform(img)
        alpha = self.tensor(alpha[:, :, 0])
        return self.imgs[idx], img, trimap, alpha

In [ ]:
ds = ImagesDataset("/home/ubuntu/data/workspace/deeplabv3_plus/people_segmentation", torch_transforms)

In [ ]:
sample = ds[4]

In [ ]:
fn, img, trimap, alpha = sample

In [ ]:
img.shape

In [ ]:
trimap.shape

In [ ]:
alpha.shape